<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/modelingsteps_initial/projects/modelingsteps/ModelingSteps_3through4_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling Steps 3 - 4

**By Neuromatch Academy**


__Content creators:__ Marius 't Hart, Megan Peters, Paul Schrater, Gunnar Blohm


__Content reviewers:__ Eric DeWitt, Tara van Viegen, Marius Pachitariu

__Production editors:__ Ella Batty, Spiros Chavlis

**Our 2021 Sponsors, including Presenting Sponsor Facebook Reality Labs**

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

----
# Step 3: Determining the basic ingredients


In [1]:
# @title Video 4: Determining basic ingredients
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="BV1Mq4y1x77s", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="XpEj-p7JkFE", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

In [2]:
# @title Example projects step 3
from ipywidgets import widgets
from IPython.display import Markdown, Math

markdown1 = r'''

## Step 3
<br>
<font size='3pt'>
We determined that we probably needed the following ingredients for our model:

* Vestibular input: *v(t)*

* Binary decision output: *d* - time dependent?

* Decision threshold: θ

* A filter (maybe running average?): *f*

* An integration mechanism to get from vestibular acceleration to sensed velocity: ∫

</font>
'''

markdown2 = '''
## Step 3
<br>
<font size='3pt'>
In order to address our question we need to design an appropriate computational data analysis pipeline. We did some brainstorming and think that we need to somehow extract the self-motion judgements from the spike counts of our neurons. Based on that, our algorithm needs to make a decision: was there self motion or not? This is a classical 2-choice classification problem. We will have to transform the raw spike data into the right input for the algorithm (spike pre-processing).

So we determined that we probably needed the following ingredients:

* spike trains *S* of 3-second trials (10ms spike bins)
* ground truth movement *m<sub>r</sub>* (real) and perceived movement *m<sub>p</sub>*
* some form of classifier *C* giving us a classification *c*
* spike pre-processing
</font>
'''

# No idea why this is necessary but math doesn't render properly without it
display(Markdown(r""))


out2 = widgets.Output()
with out2:
  display(Markdown(markdown2))


out1 = widgets.Output()
with out1:
  display(Markdown(markdown1))

out = widgets.Tab([out1, out2])
out.set_title(0, 'Computational Model')
out.set_title(1, 'Data Analysis')

display(out)

## Determine your basic ingredients 

This will allow you to think deeper about what your modeling project will need. It's a crucial step before you can formulate hypotheses because you first need to understand what your modeling approach will need. There are 2 aspects you want to think about:

<br>

1.   What parameters / hyperparameters / variables are needed?
      * Constants?
      * Do they change over space, time, conditions…?
      * What details can be omitted?
      * Constraints, initial conditions?
      * Model inputs / outputs?
2.   Variables needed to describe the process to be modelled?
      * Brainstorming!
      * What can be observed / measured?  latent variables?    
      * Where do these variables come from?
      * Do any abstract concepts need to be instantiated as variables?
        * E.g., value, utility, uncertainty, cost, salience, goals, strategy, plant, dynamics
        * Instantiate them so that they relate to potential measurements!

This is a step where your prior knowledge and intuition is tested. You want to end up with an inventory of *specific* concepts and/or interactions that need to be instantiated. 

**Make sure to avoid the pitfalls!**
<details>
<summary>Click here for a recap on pitfalls</summary>

I’m experienced, I don’t need to think about ingredients anymore
  <ul>
  <li>Or so you think …</li>
  </ul>

I can’t think of any ingredients
  <ul>
  <li>Think about the potential experiment. What are your stimuli? What parameters? What would you control? What do you measure?</li>
  </ul>

I have all inputs and outputs
  <ul>
  <li>Good! But what will link them? Thinking about that will start shaping your model and hypotheses</li>
  </ul>

I can’t think of any links (= mechanisms)
  <ul>
  <li>You will acquire a library of potential mechanisms as you keep modeling and learning</li>
  <li>But the literature will often give you hints through hypotheses</li>
  <li>If you still can't think of links, then maybe you're missing ingredients?</li>
  </ul>
</details>

----
# Step 4: Formulating specific, mathematically defined hypotheses



In [3]:
# @title Video 5: Formulating a hypothesis
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="BV1fh411h7aX", width=854, height=480, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="nHXMSXLcd9A", width=854, height=480, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)

In [4]:
# @title Example projects step 4
from ipywidgets import widgets
from IPython.display import Markdown

# Not writing in latex because that didn't render in jupyterbook

markdown1 = r'''

## Step 4
<br>
<font size='3pt'>
Our main hypothesis is that the strength of the illusion has a linear relationship to the amplitude of vestibular noise.

Mathematically, this would write as

<div align="center">
<em>S</em> = <em>k</em> ⋅ <em>N</em>
</div>

where *S* is the illusion strength and *N* is the noise level, and *k* is a free parameter.
>we could simply use the frequency of occurance across repetitions as the "strength of the illusion"

We would get the noise as the standard deviation of *v(t)*, i.e.

<div align="center">
<em>N</em> = <b>E</b>[<em>v(t)</em><sup>2</sup>],
</div>

 where **E** stands for the expected value.

Do we need to take the average across time points?
> doesn't really matter because we have the generative process, so we can just use the σ that we define
</font>
'''

markdown2 = '''
## Step 4
<br>

<font size='3pt'>
We think that noise in the signal drives whether or not people perceive self motion. Maybe the brain uses the strongest signal at peak acceleration to decide on self motion, but we actually think it is better to accumulate evidence over some period of time. We want to test this. The noise idea also means that when the signal-to-noise ratio is higher, the brain does better, and this would be in the faster acceleration condition. We want to test this too.

We came up with the following hypotheses focussing on specific details of our overall research question:

* Hyp 1: Accumulated vestibular spike rates explain self-motion judgements better than average spike rates around peak acceleration.
* Hyp 2: Classification performance should be better for faster vs slower self-motion.

> There are many other hypotheses you could come up with, but for simplicity, let's go with those.

Mathematically, we can write our hypotheses as follows (using our above ingredients):
* Hyp 1: **E**(c<sub>accum</sub>) > **E**(c<sub>win</sub>)
* Hyp 2: **E**(c<sub>fast</sub>) > **E**(c<sub>slow</sub>)

Where **E** denotes taking the expected value (in this case the mean) of its argument: classification outcome in a given trial type.
</font>
'''

# No idea why this is necessary but math doesn't render properly without it
display(Markdown(r""))

out2 = widgets.Output()
with out2:
  display(Markdown(markdown2))

out1 = widgets.Output()
with out1:
  display(Markdown(markdown1))

out = widgets.Tab([out1, out2])
out.set_title(0, 'Computational Model')
out.set_title(1, 'Data Analysis')

display(out)

## Formulating your hypothesis 

Once you have your question and goal lines up, you have done a literature review and you have thought about ingredients needed for your model, you're now ready to start thinking about *specific* hypotheses.

Formulating hypotheses really consists of two consecutive steps:



1.   You think about the hypotheses in words by relating ingredients identified in Step 3
      * What is the model mechanism expected to do? 
      * How are different parameters expected to influence model results?
2.   You then express these hypotheses in mathematical language by giving the ingredients identified in Step 3 specific variable names. 
      * Be explicit, e.g., $y(t)=f(x(t), k)$ but $z(t)$ doesn’t influence $y$


There are also "structural hypotheses" that make assumptions on what model components you hypothesize will be crucial to capture the phenomenon at hand. 

**Important:** Formulating the hypotheses is the last step before starting to model. This step determines the model approach and ingredients. It provides a more detailed description of the question / goal from Step 1. The more precise the hypotheses, the easier the model will be to justify. 

**Make sure to avoid the pitfalls!**
<details>
<summary>Click here for a recap on pitfalls</summary>

I don’t need hypotheses, I will just play around with the model
  <ul>
  <li>Hypotheses help determine and specify goals. You can (and should) still play…</li>
  </ul>

My hypotheses don’t match my question (or vice versa)
  <ul>
  <li>This is a normal part of the process!</li>
  <li>You need to loop back to Step 1 and revisit your question / phenomenon / goals</li>
  </ul>

I can’t write down a math hypothesis
  <ul>
  <li>Often that means you lack ingredients and/or clarity on the hypothesis</li>
  <li>OR: you have a “structural” hypothesis, i.e. you expect a certain model component to be crucial in explaining the phenomenon / answering the question</li>
  </ul>
</details>